In [2]:
import sys, os, re, csv, codecs 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt, seaborn as sns
import string
import nltk
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import stopwords 
eng_stopwords = set(stopwords.words("english"))
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
nltk.download('stopwords')
nltk.download("punkt")
nltk.download("words")
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
#from autocorrect import spell
lem = WordNetLemmatizer()
nltk.download('wordnet')
from nltk.corpus import words
word_list = words.words()
import json
with open('../input/words.json') as json_data:
    profane_words = json.load(json_data)
    
#need to git clone https://github.com/zacanger/profane-words
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()
#from textblob import TextBlob
#from textblob.sentiments import NaiveBayesAnalyzer
nltk.download('averaged_perceptron_tagger')
nltk.download('movie_reviews')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rdufogsa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rdufogsa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/rdufogsa/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rdufogsa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rdufogsa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/rdufogsa/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
path = "../input/"
embedding_file = f'{path}glove.6B.50d.txt'
train_file = f'{path}train.csv'
test_file = f'{path}test.csv'

In [4]:
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
train["comment_text"].fillna("unknown", inplace = True)
test["comment_text"].fillna("unknown", inplace = True)
train.shape

(159571, 8)

In [5]:
text = pd.DataFrame(train["comment_text"])
toxic = pd.DataFrame(train["toxic"])
severe_toxic = pd.DataFrame(train["severe_toxic"])
obscene_toxic = pd.DataFrame(train["obscene"])
threat = pd.DataFrame(train["obscene"])
insult = pd.DataFrame(train["insult"])
identity_hate = pd.DataFrame(train["identity_hate"])
#text.head()


In [6]:
test = test.fillna("")

In [7]:
labeled = train[(train["toxic"] != 0) | (train["severe_toxic"] != 0) | 
              (train["obscene"] != 0) | (train["threat"] != 0) | 
              (train["insult"] != 0) | (train["identity_hate"] != 0)]


### Comment Cleaning 

In [8]:
APPO = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" }


### Test

In [ ]:
polarity = test["comment_text"].apply(lambda x: sent_analyzer.polarity_scores(x))
#negative polarity percentage of the text
test["neg_polarity"] = polarity.apply(lambda x: x["neg"])
#neutral polarity percentage 
test["neutral_polarity"] = polarity.apply(lambda x: x["neu"])
#positive polarity percentage
test["positive_polarity"] = polarity.apply(lambda x: x["pos"])
#sum of sentimental ratings
test["compound_polarity"] = polarity.apply(lambda x: x["compound"])

In [ ]:
#Number of sentences
test["count_sent"] = test["comment_text"].apply(lambda x: len(re.findall("\n", str(x))) + 1)
#number of words
test["count_word"] = test["comment_text"].apply(lambda x: len(str(x).split()))
#number of unique words
test["count_unique_word"] = test["comment_text"].apply(lambda x: len(set(str(x).split())))
#number of characters
test["count_char"] = test["comment_text"].apply(lambda x: len(str(x)))
#number of punctuations
test["count_punctuations"] = test["comment_text"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))#number of uppercase letters
#number of uppercase letters
test["count_words_upper"] = test["comment_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
#number of titles
test["count_words_title"] = test["comment_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
#number of useless words
test["count_stopwords"] = test["comment_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
#mean length of words 
test["mean_word_len"] = test["comment_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
#unique word percent
test['word_unique_percent']=test['count_unique_word']*100/test['count_word']
#punctuation percent
test['punct_percent']=test['count_punctuations']*100/test['count_word']
#train["cleaned"] = train["comment_text"].apply(lambda x: clean(x))


In [ ]:
test.head()

### Cleaned

In [ ]:
def misspelled_and_swears(comment):
    words = nltk.word_tokenize(comment)
    swear_counts = sum([words[i] in profane_words for i in range(len(words))])
    misspelled_prop = 1 - sum([words[i] in word_list for i in range(len(words))])/len(words)
    return [misspelled_prop, 1 if swear_counts > 0 else 0, swear_counts]

In [10]:
len(test)

153164

In [12]:
misspelled_profane = test["comment_text"].iloc[].apply(lambda x: misspelled_and_swears(x))
#feel free to run the rows 

SyntaxError: invalid syntax (<ipython-input-12-0af54c467f2d>, line 1)

In [ ]:
test = misspelled_profane.to_csv("comment_and_features_test.csv")

0       [0.308641975308642, 1, 5]
1                     [0.5, 0, 0]
2      [0.8333333333333334, 0, 0]
3     [0.20454545454545459, 0, 0]
4     [0.33333333333333337, 0, 0]
5     [0.16666666666666663, 0, 0]
6      [0.3529411764705882, 0, 0]
7                   [0.375, 0, 0]
8      [0.4201680672268907, 1, 1]
9       [0.326530612244898, 0, 0]
10                   [0.24, 1, 1]
11     [0.4838709677419355, 1, 1]
12    [0.49230769230769234, 0, 0]
13                    [0.5, 0, 0]
14      [0.352112676056338, 0, 0]
15     [0.3157894736842105, 0, 0]
16    [0.19999999999999996, 0, 0]
17                [0.40625, 0, 0]
18    [0.30666666666666664, 0, 0]
19                  [0.875, 0, 0]
20    [0.33333333333333337, 0, 0]
21     [0.4545454545454546, 1, 1]
22     [0.3913043478260869, 0, 0]
23    [0.24444444444444446, 0, 0]
24    [0.22857142857142854, 1, 1]
25    [0.36871508379888274, 0, 0]
26     [0.4516129032258065, 0, 0]
27    [0.38888888888888884, 0, 0]
28    [0.22499999999999998, 0, 0]
29    [0.23076